In [ ]:
!pip install wheel setuptools pip --upgrade
!pip install --upgrade openai

  Using cached setuptools-75.1.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached setuptools-75.1.0-py3-none-any.whl (1.2 MB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 71.0.4
    Uninstalling setuptools-71.0.4:
      Successfully uninstalled setuptools-71.0.4
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [ ]:
import pandas as pd
import random
from openai import OpenAI
import time
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
credit_df = pd.read_csv('credit_new.csv')
credit_df = credit_df.fillna('Unknown')
credit_df.drop('Unnamed: 0', axis=1, inplace=True)

credit_df['SeriousDlqin2yrs'] = credit_df['SeriousDlqin2yrs'].replace(0, 'will not experience financial distress')
credit_df['SeriousDlqin2yrs'] = credit_df['SeriousDlqin2yrs'].replace(1, 'will experience financial distress')

# Remove the column and store it
column_to_move = credit_df.pop('SeriousDlqin2yrs')

# Add the column to the end of the DataFrame
credit_df['SeriousDlqin2yrs'] = column_to_move

print(credit_df.head(5))

tuning_set = credit_df.sample(n = 800, random_state = 1)
percentage_yes = (tuning_set['SeriousDlqin2yrs'] == 'will experience financial distress').mean()
print(tuning_set['SeriousDlqin2yrs'].unique())
print(percentage_yes)

row_series = tuning_set.iloc[0][:-1]

all_features = ''
for feature, value in row_series.items():
    all_features += feature + ', '

print(all_features)

selected_rows_1 = credit_df.loc[credit_df['SeriousDlqin2yrs'] == 'will experience financial distress'].head(5)

selected_rows_2 = credit_df.loc[credit_df['SeriousDlqin2yrs'] == 'will not experience financial distress'].head(15)

# Combine the two DataFrames into a new DataFrame
combined_df = pd.concat([selected_rows_1, selected_rows_2], ignore_index=True)

samples = []
targets = []
for i in range( len(combined_df) ):
    row_series = combined_df.iloc[i][:-1]
    target = combined_df.iloc[i]['SeriousDlqin2yrs']

    overall_feature_levels = ''
    for feature, value in row_series.items():
        feature_level = (f"Feature: {feature}, Value: {value}")
        overall_feature_levels += feature_level + '; '

    samples.append(overall_feature_levels)
    targets.append(target)

print(samples)
print(targets)

In [ ]:
API_KEY = ''
client = OpenAI(api_key = API_KEY)
model_id = 'gpt-3.5-turbo-0125'

APE_prompt = f"""
You are given a set of feature-value pairs that represents a person's financial profile. Your task is to propose a creative, detailed, and step-by-step algorithm to reformulate and enrich (make more informative) this profile. The goal is of the algorithm is to perform a thorough data engineering among the feature values, so that it is easier for a LLM to distinguish whether the person is likely to experience financial distrees in the next 2 years. Below are some sample financial profiles as refrences.

Examples:
- Financial Profile: '{samples[0]}'; Will the person experience financial distress in the next two years: '{targets[0]}'
- Financial Profile: '{samples[1]}'; Will the person experience financial distress in the next two years: '{targets[1]}'
- Financial Profile: '{samples[5]}'; Will the person experience financial distress in the next two years: '{targets[5]}'
- Financial Profile: '{samples[6]}'; Will the person experience financial distress in the next two years: '{targets[6]}'
- Financial Profile: '{samples[7]}'; Will the person experience financial distress in the next two years: '{targets[7]}'
- Financial Profile: '{samples[8]}'; Will the person experience financial distress in the next two years: '{targets[8]}'
- Financial Profile: '{samples[9]}'; Will the person experience financial distress in the next two years: '{targets[9]}'
- Financial Profile: '{samples[10]}'; Will the person experience financial distress in the next two years: '{targets[10]}'
- Financial Profile: '{samples[11]}'; Will the person experience financial distress in the next two years: '{targets[11]}'
- Financial Profile: '{samples[12]}'; Will the person experience financial distress in the next two years: '{targets[12]}'

For each step of the algorithm, number and then start it on a new line; you must start each step with '###------- Step 1: ' The proposed algorithm will later be submitted to a LLM for processing.
Important: Do NOT refer to any external database; Do NOT perform the item counting. Do NOT perform normalization. Do NOT perform PCA. Do NOT perform standardization; Do NOT perform one-hot encoding; Do NOT perform vector generations. Do NOT perform similarity checking. ONLY propose things that a LLM can do on its own!
"""

prompts = []
prompt_performances = {}
for i in range(7):

    completion = client.chat.completions.create(
        model = model_id, temperature = 1.0,

        messages=[{"role": "system", "content": "Please come up with a step-by-step, very detailed, clear, and novel algorithm for reformulating and enriching a person's financial profile. Return the steps ONLY!!! NO more than 6 steps. You MUST NOT directly include whether the person will expereince financial distress in the next 2 years within the generated text!!!"},
                    {"role": "user", "content": APE_prompt}],
        timeout = 1200)

    candidate_prompt = completion.choices[0].message.content
    prompts.append(candidate_prompt)
    print(candidate_prompt)
    print()


few_shot_pairs = f'''
- Financial Profile: '{samples[0]}'; Will the person experience financial distress in the next two years: '{targets[0]}'
- Financial Profile: '{samples[1]}'; Will the person experience financial distress in the next two years: '{targets[1]}'
- Financial Profile: '{samples[2]}'; Will the person experience financial distress in the next two years: '{targets[2]}'
- Financial Profile: '{samples[3]}'; Will the person experience financial distress in the next two years: '{targets[3]}'
- Financial Profile: '{samples[12]}'; Will the person experience financial distress in the next two years: '{targets[12]}'
'''

###------- Step 1: Identify missing values in the financial profile such as 'Unknown' or 'NaN'.
###------- Step 2: Calculate the credit utilization ratio based on the 'RevolvingUtilizationOfUnsecuredLines' feature.
###------- Step 3: Determine the person's creditworthiness based on the number of times they were past due in different time frames.
###------- Step 4: Analyze the person's debt-to-income ratio by considering the 'DebtRatio' and 'MonthlyIncome' features.
###------- Step 5: Assess the stability and diversification of the person's finances by looking at the number of open credit lines, real estate loans, and dependents.
###------- Step 6: Evaluate the person's financial health based on their age and the overall distribution of the financial features in the profile.

###------- Step 1: Identify missing values in the financial profile and categorize them.
###------- Step 2: For missing values, impute them with appropriate statistical metrics such as mean, median, or mode based o

In [ ]:
import re

system_msg = 'Please serve as a binary classifier on whether the person is likely to experience financial distress within the next 2 years, based on his/her current financial profile.'

for candidate_prompt in prompts:
    print(candidate_prompt)
    print()

    # Regular expression to split the text into individual algorithms
    algorithm_pattern = r"###------- Step \d: [\w\s]+"

    # Split the text based on the pattern
    split_text = re.split(algorithm_pattern, candidate_prompt)

    # Extract the algorithm headers (for identification)
    headers = re.findall(algorithm_pattern, candidate_prompt)

    # Removing the first empty string from the split if exists (because of the split at the start)
    split_text = [t.strip() for t in split_text if t.strip()]

    # Create a dictionary where each algorithm is stored separately
    algorithms = {headers[i]: split_text[i] for i in range(len(headers))}

    # Display each algorithm separately
    steps = []
    for header, content in algorithms.items():
        steps.append( [header, content] )
        # print(f"{header}:\n{content}\n")


    right_count_pos = 0
    right_count_neg = 0
    compressed_right_count_pos = 0
    compressed_right_count_neg = 0
    total_pos = 0
    total_neg = 0
    total = 0

    # you may also set range smaller for a subset
    for i in range( len(tuning_set) ):
        row_series = tuning_set.iloc[i][:-1]
        target = tuning_set.iloc[i]['SeriousDlqin2yrs']

        overall_feature_levels = ''
        for feature, value in row_series.items():
            feature_level = (f"Feature: {feature}, Value: {value}")
            overall_feature_levels += feature_level + '; '


        # original_prompt = (
        #     "You are given a person's financial profile with each feature-value pair presented."
        #     "Please determine whether the person is likely to experience a financial distress within the next 2 years. First write a step-by-step reasoning paragraph, and then output either 'will experience financial distress' or 'will not experience financial distress' strictly, in lower case!!!"
        #     "Below is the person's financial profile: \n\n"
        #     f"{overall_feature_levels}"
        # )

        # original_prompt = (
        #     f"Given the following person's financial profile: \n\n {overall_feature_levels};\n\n"
        #     f"Please determine whether the person is likely to experience a financial distress within the next 2 years. Do NOT explain anything, just output either 'will experience financial distress' or 'will not experience financial distress' strictly, in lower case!!!"
        # )

        original_prompt = (
            f"Given the following person's financial profile: \n\n {overall_feature_levels};\n\n"
            f'You are also given several examples to learn from: \n\n {few_shot_pairs};\n\n'
            f"Please determine whether the person is likely to experience a financial distress within the next 2 years. Do NOT explain anything, just output either 'will experience financial distress' or 'will not experience financial distress' strictly, in lower case!!!"
        )

        completion = client.chat.completions.create(
                model = model_id, temperature = 0, seed = 0,
                messages=[{"role": "system", "content": system_msg},
                            {"role": "user", "content": original_prompt }],
                timeout = 1200)

        original_pred = completion.choices[0].message.content
        # print(original_pred)
        # print()

        total += 1
        if target == 'will experience financial distress':
            total_pos += 1
        else:
            total_neg += 1

        if target in original_pred:
            if (target == 'will experience financial distress'):
                right_count_pos += 1
            if (target == 'will not experience financial distress'):
                right_count_neg += 1



        overall_with_steps = "Original person's financial profile: " + overall_feature_levels + '\n\n'
        for i in range(len(algorithms)):

            reformulation_prompt = f"Please thoroughly enrich and reformulate the person's financial profile based on the following instruction:\n\n{steps[i][0]}\n{steps[i][1]}\n\nprofile to enrich and reformulate: {overall_with_steps}\n\nYou MUST return the reformulation of the profile ONLY!!!"

            completion = client.chat.completions.create(
                    model = model_id, temperature = 0.0, max_tokens = 512,

                    messages=[{"role": "system", "content": "Please enrich and reformulate the person's profile to be extremely informative and detailed for LLM to better recognize the individuals that will be in financial distress within the next 2 years. You are allowed to fill in unspecified information based on your domain expertise!"},
                                {"role": "user", "content": reformulation_prompt}],
                    timeout = 1200)

            reformulated_history = completion.choices[0].message.content
            # print('Step ' + str(i) + ' Results: ')
            # print(reformulated_history)
            overall_with_steps += steps[i][0] + '\n' + steps[i][1] + reformulated_history + '\n\n'
            # print()
            # print()

        print(overall_with_steps)
        print()


        # ade_prompt = (
        #     "You are given a person's financial profile with each feature-value pair presented."
        #     "Please determine whether the person is likely to experience a financial distress within the next 2 years. First write a procedual reasoning paragraph, and then output either 'will experience financial distress' or 'will not experience financial distress' strictly, in lower case!!!"
        #     "Below is the person's financial profile: \n\n"
        #     f"{overall_with_steps}"
        # )

        ade_prompt = (
            f"Given the following person's financial profile: \n\n {overall_with_steps};\n\n"
            f'You are also given several examples to learn from: \n\n {few_shot_pairs};\n\n'
            f"Please determine whether the person is likely to experience a financial distress within the next 2 years. Do NOT explain anything, just output either 'will experience financial distress' or 'will not experience financial distress' strictly, in lower case!!!"
        )

        # ade_prompt = (
        #     f"Given the following person's financial profile: \n\n {overall_with_steps};\n\n"
        #     f"Please determine whether the person is likely to experience a financial distress within the next 2 years. Do NOT explain anything, just output either 'will experience financial distress' or 'will not experience financial distress' strictly, in lower case!!!"
        # )

        compressed_completion = client.chat.completions.create(
                model = model_id, temperature = 0, seed = 0,
                messages=[{"role": "system", "content": system_msg},
                            {"role": "user", "content": ade_prompt}],
                timeout = 1200)

        compressed_pred = compressed_completion.choices[0].message.content
        # end


        if target in compressed_pred:
            if (target == 'will experience financial distress'):
                compressed_right_count_pos += 1
            if (target == 'will not experience financial distress'):
                compressed_right_count_neg += 1


        # print(converted_feature_levels)
        print(right_count_pos, right_count_neg)
        print(compressed_right_count_pos, compressed_right_count_neg)
        print()

        if total % 100 == 0 or total == credit_df.shape[0]:
            accuracy = (right_count_pos/total_pos) * 0.5 + (right_count_neg/total_neg) * 0.5
            compressed_accuracy = (compressed_right_count_pos/total_pos) * 0.5 + (compressed_right_count_neg/total_neg) * 0.5
            print('Accuracy: ', accuracy)
            print('Compressed Accuracy: ', compressed_accuracy)
            print()

            if compressed_accuracy <= accuracy:
                break

    prompt_performances[candidate_prompt] = [ (right_count_pos/total_pos) * 0.5 + (right_count_neg/total_neg) * 0.5, (compressed_right_count_pos/total_pos) * 0.5 + (compressed_right_count_neg/total_neg) * 0.5 ]
    print()

